In [1]:
import os
import sys
import platform
import subprocess
import multiprocessing
import ctypes
import time

REPORT = {}

# ------------------------------
# 1️⃣ Hardware Check
# ------------------------------

def check_cpu():
    cpu_info = platform.processor()
    machine = platform.machine()

    neon_supported = False
    try:
        with open("/proc/cpuinfo", "r") as f:
            data = f.read().lower()
            if "neon" in data or "asimd" in data:
                neon_supported = True
    except:
        pass

    REPORT["CPU Architecture"] = machine
    REPORT["Processor"] = cpu_info
    REPORT["NEON Supported"] = neon_supported


# ------------------------------
# 2️⃣ OS Check
# ------------------------------

def check_os():
    try:
        with open("/etc/os-release") as f:
            os_info = f.read()
    except:
        os_info = "Unknown"

    REPORT["OS Info"] = os_info.split("\n")[0]


# ------------------------------
# 3️⃣ Python Check
# ------------------------------

def check_python():
    REPORT["Python Version"] = sys.version
    major, minor = sys.version_info[:2]

    if (major == 3 and minor in [9, 10, 11]):
        REPORT["Python Compatible"] = True
    else:
        REPORT["Python Compatible"] = False


# ------------------------------
# 4️⃣ System Library Check
# ------------------------------

def check_library(libname):
    try:
        ctypes.CDLL(libname)
        return True
    except:
        return False


def check_system_libs():
    libs = [
        "libarm_compute.so",
        "libarm_compute_core.so",
        "libarm_delegate.so",
        "libopenblas.so",
    ]

    for lib in libs:
        REPORT[f"{lib} Available"] = check_library(lib)


# ------------------------------
# 5️⃣ TFLite Runtime Check
# ------------------------------

def check_tflite():
    try:
        import tflite_runtime.interpreter as tflite
        REPORT["tflite-runtime Installed"] = True
    except:
        REPORT["tflite-runtime Installed"] = False


# ------------------------------
# 6️⃣ Threading Check
# ------------------------------

def check_threading():
    REPORT["CPU Cores"] = multiprocessing.cpu_count()

    try:
        import threading
        REPORT["Threading Supported"] = True
    except:
        REPORT["Threading Supported"] = False


# ------------------------------
# 7️⃣ Delegate Load Test
# ------------------------------

def test_delegate():
    try:
        import tflite_runtime.interpreter as tflite
        delegate = tflite.load_delegate("libarm_delegate.so")
        REPORT["Arm Delegate Loadable"] = True
    except:
        REPORT["Arm Delegate Loadable"] = False


# ------------------------------
# 8️⃣ Simple Parallel Pipeline Test
# ------------------------------

def heavy_task(n):
    s = 0
    for i in range(10_000_000):
        s += i
    return s


def test_parallel_pipeline():
    start = time.time()
    processes = []
    for _ in range(2):
        p = multiprocessing.Process(target=heavy_task, args=(1,))
        p.start()
        processes.append(p)

    for p in processes:
        p.join()

    duration = time.time() - start
    REPORT["Parallel Execution Test (2 processes sec)"] = round(duration, 3)


# ------------------------------
# 9️⃣ INT8 Model Compatibility Check
# ------------------------------

def check_model_int8(model_path="model_int8.tflite"):
    try:
        import tflite_runtime.interpreter as tflite
        interpreter = tflite.Interpreter(model_path=model_path)
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()

        dtype = input_details[0]['dtype']
        REPORT["Model Found"] = True
        REPORT["Model Input Dtype"] = str(dtype)

        if "int8" in str(dtype):
            REPORT["INT8 Quantized"] = True
        else:
            REPORT["INT8 Quantized"] = False

    except Exception as e:
        REPORT["Model Found"] = False
        REPORT["Model Error"] = str(e)


# ------------------------------
# 🔟 Run All Checks
# ------------------------------

def main():
    print("\n===== KleidiAI + TFLite System Audit =====\n")

    check_cpu()
    check_os()
    check_python()
    check_system_libs()
    check_tflite()
    check_threading()
    test_delegate()
    test_parallel_pipeline()
    check_model_int8()

    for k, v in REPORT.items():
        print(f"{k}: {v}")

    print("\n===== Audit Complete =====\n")


if __name__ == "__main__":
    main()



===== KleidiAI + TFLite System Audit =====

CPU Architecture: AMD64
Processor: AMD64 Family 25 Model 80 Stepping 0, AuthenticAMD
NEON Supported: False
OS Info: Unknown
Python Version: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
Python Compatible: True
libarm_compute.so Available: False
libarm_compute_core.so Available: False
libarm_delegate.so Available: False
libopenblas.so Available: False
tflite-runtime Installed: False
CPU Cores: 16
Threading Supported: True
Arm Delegate Loadable: False
Parallel Execution Test (2 processes sec): 0.159
Model Found: False
Model Error: No module named 'tflite_runtime'

===== Audit Complete =====

